In [1]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import numpy.ma as ma

In [4]:
file_path='./data/sanjioil.nc'

In [5]:
ds=nc.Dataset(file_path)

In [6]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    standard_name_vocabulary: CF-1.6
    featureType: trajectory
    history: Created 2019-08-01 11:42:45.072000
    source: Output from simulation with OpenDrift
    model_url: https://github.com/OpenDrift/opendrift
    opendrift_class: OpenOil
    opendrift_module: opendrift.models.openoil
    readers: ['E:\\project\\oilspill_rescue\\curr_wind\\curr0_test.nc', 'E:\\project\\oilspill_rescue\\curr_wind\\wind0_test.nc', 'basemap_landmask']
    time_coverage_start: 2018-01-14 22:20:00
    time_step_calculation: 0:30:00
    time_step_output: 1:00:00
    config_seed:oil_type: MEDIUM (brage)
    config_seed:ocean_only: True
    config_general:use_basemap_landmask: True
    config_general:basemap_resolution: h
    config_general:coastline_action: stranding
    config_general:time_step_minutes: 60
    config_general:time_step_output_minutes: None
    config_drift:max_age_seconds: None

In [7]:
ds.variables.keys()

odict_keys(['trajectory', 'time', 'age_seconds', 'interfacial_area', 'mass_evaporated', 'water_fraction', 'y_wind', 'sea_ice_area_fraction', 'density', 'lon', 'oil_film_thickness', 'x_sea_water_velocity', 'bulltime', 'x_wind', 'age_emulsion_seconds', 'sea_surface_wave_stokes_drift_y_velocity', 'status', 'viscosity', 'sea_floor_depth_below_sea_level', 'lat', 'mass_dispersed', 'land_binary_mask', 'sea_surface_wave_significant_height', 'sea_water_temperature', 'mass_oil', 'age_exposure_seconds', 'sea_surface_wave_stokes_drift_x_velocity', 'y_sea_water_velocity', 'z', 'wind_drift_factor', 'fraction_evaporated'])

In [8]:
# 查看维度
ds.dimensions.keys()

odict_keys(['trajectory', 'time'])

只有两个维度，`trajectory`与`time`

## 2 读取维度相关

### 获取轨迹

In [9]:
len(ds['trajectory'][:])

2958

In [10]:
ds['trajectory'][:]

masked_array(data=[   1,    2,    3, ..., 2956, 2957, 2958],
             mask=False,
       fill_value=999999)

In [11]:
# 找到其中一个
ds['trajectory'][:][1]

2

### 获取time  
时间共73个

In [12]:
len(ds['time'][:])

73

In [6]:
ds['time']

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    units: seconds since 1970-01-01 00:00:00
    standard_name: time
    long_name: time
unlimited dimensions: 
current shape = (25,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [28]:
ds['time'][:]

masked_array(data=[1.4691348e+09, 1.4691366e+09, 1.4691384e+09,
                   1.4691402e+09, 1.4691420e+09, 1.4691438e+09,
                   1.4691456e+09, 1.4691474e+09, 1.4691492e+09,
                   1.4691510e+09, 1.4691528e+09, 1.4691546e+09,
                   1.4691564e+09, 1.4691582e+09, 1.4691600e+09,
                   1.4691618e+09, 1.4691636e+09, 1.4691654e+09,
                   1.4691672e+09, 1.4691690e+09, 1.4691708e+09,
                   1.4691726e+09, 1.4691744e+09, 1.4691762e+09,
                   1.4691780e+09],
             mask=False,
       fill_value=1e+20)

## 3 读取变量

### 读取经纬度

In [13]:
ds['lat']

<class 'netCDF4._netCDF4.Variable'>
float32 lat(trajectory, time)
    coordinates: lat lon time
    units: degrees_north
    standard_name: latitude
    long_name: latitude
unlimited dimensions: 
current shape = (2958, 73)
filling on, default _FillValue of 9.969209968386869e+36 used

In [14]:
ds['lon']

<class 'netCDF4._netCDF4.Variable'>
float32 lon(trajectory, time)
    coordinates: lat lon time
    units: degrees_east
    standard_name: longitude
    long_name: longitude
unlimited dimensions: 
current shape = (2958, 73)
filling on, default _FillValue of 9.969209968386869e+36 used

对73个时刻的经纬度求平均值  
如果别的案例里有粒子靠岸的话，求平均的时候要把靠岸的粒子剔除掉

In [16]:
ds['time'].size

73

In [29]:
len(ds['lat'][:].T[0])

2958

按照时间获取所有的轨迹点 
> 注意需要对某一个变量的矩阵先进行 转置 ，再按照下标读取（长度73个）

In [30]:
ds['lat'][:].T

masked_array(
  data=[[28.36937141418457, 28.369325637817383, 28.369295120239258, ...,
         --, --, --],
        [28.36955451965332, 28.373537063598633, 28.3829288482666, ...,
         --, --, --],
        [--, 28.39068031311035, --, ..., --, --, --],
        ...,
        [--, 28.44026756286621, --, ..., --, --, --],
        [--, 28.429256439208984, --, ..., --, --, --],
        [--, 28.45772933959961, --, ..., 28.367712020874023,
         28.379928588867188, 28.375484466552734]],
  mask=[[False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [ True, False,  True, ...,  True,  True,  True],
        ...,
        [ True, False,  True, ...,  True,  True,  True],
        [ True, False,  True, ...,  True,  True,  True],
        [ True, False,  True, ..., False, False, False]],
  fill_value=9.96921e+36,
  dtype=float32)

In [31]:
ds['lat'][:].T[0]

masked_array(data=[28.36937141418457, 28.369325637817383,
                   28.369295120239258, ..., --, --, --],
             mask=[False, False, False, ...,  True,  True,  True],
       fill_value=9.96921e+36,
            dtype=float32)

根据时间维度遍历经纬度，并取平均值

In [37]:
lat_arr=[]
for temp in range(ds['time'].size):
    lat_arr.append(ds['lat'][:].T[temp].mean())
    print(f"{temp}:{ds['lat'][:].T[temp].mean()}")    

0:28.369285946800595
1:28.37451171875
2:28.37786287006579
3:28.38084746570122
4:28.381984958400974
5:28.379158266129032
6:28.37642568912146
7:28.37551249354339
8:28.373883928571427
9:28.37420215482026
10:28.37706389925373
11:28.383863772199454
12:28.39144736842105
13:28.399444361575178
14:28.405360630580358
15:28.409958881578948
16:28.41013702632874
17:28.407168151285045
18:28.401532823542404
19:28.395347829564848
20:28.390202616869917
21:28.38865966796875
22:28.391369047619047
23:28.398504273504273
24:28.410381301440328
25:28.425336657072368
26:28.442340569338423
27:28.457021640528904
28:28.467798892920616
29:28.475530998851895
30:28.479506277746946
31:28.481130326321466
32:28.48226117484177
33:28.483989197530864
34:28.48824996871872
35:28.49610899390244
36:28.50712425595238
37:28.51806144679132
38:28.530382530968467
39:28.5404403659612
40:28.54443736587983
41:28.545134883598994
42:28.539387224938874
43:28.531386843152866
44:28.522470121473354
45:28.51338341346154
46:28.50525140977443

In [38]:
lat_arr

[28.369285946800595,
 28.37451171875,
 28.37786287006579,
 28.38084746570122,
 28.381984958400974,
 28.379158266129032,
 28.37642568912146,
 28.37551249354339,
 28.373883928571427,
 28.37420215482026,
 28.37706389925373,
 28.383863772199454,
 28.39144736842105,
 28.399444361575178,
 28.405360630580358,
 28.409958881578948,
 28.41013702632874,
 28.407168151285045,
 28.401532823542404,
 28.395347829564848,
 28.390202616869917,
 28.38865966796875,
 28.391369047619047,
 28.398504273504273,
 28.410381301440328,
 28.425336657072368,
 28.442340569338423,
 28.457021640528904,
 28.467798892920616,
 28.475530998851895,
 28.479506277746946,
 28.481130326321466,
 28.48226117484177,
 28.483989197530864,
 28.48824996871872,
 28.49610899390244,
 28.50712425595238,
 28.51806144679132,
 28.530382530968467,
 28.5404403659612,
 28.54443736587983,
 28.545134883598994,
 28.539387224938874,
 28.531386843152866,
 28.522470121473354,
 28.51338341346154,
 28.505251409774438,
 28.500045955882353,
 28.4979586693